In [1]:
using Random
using Distributions

In [2]:
abstract type AbstractBlk end

struct Minor
    label::String
end

struct Genesis <: AbstractBlk end

struct Blk <: AbstractBlk
    index::Int
    minor::Minor
    prev::AbstractBlk
end

function Base.show(io::IO, b::Genesis)
    Base.show(io, 0)
end

function Base.show(io::IO, b::Blk)
    Base.show(io, (b.index, b.minor.label))
end

struct Queue
    sender::Minor
    receiver::Minor
    elem::Vector{AbstractBlk}
    
    Queue(sender, receiver) = new(sender, receiver, Vector{AbstractBlk}())
end

In [3]:
getid(b::Blk) = b.index
getid(b::Genesis) = 0

getid (generic function with 2 methods)

In [4]:
getreceiver(q::Queue) = q.receiver

function receiveblk!(q::Queue, i::Int)::AbstractBlk
    b = q.elem[i]
    deleteat!(q.elem, i)
    b
end

getlength(q::Queue) = length(q.elem)

getlength (generic function with 1 method)

In [5]:
function sendblk!(q::Queue, b::AbstractBlk)
    push!(q.elem, b)
end

sendblk! (generic function with 1 method)

In [6]:
function event_genBlk!(x::Minor, workblk::Dict{Minor,AbstractBlk}, queue::Vector{Queue}, blocks::Vector{Blk})
    prev = workblk[x]
    id = getid(prev) + 1
    b = Blk(id, x, prev)
    push!(blocks, b)
    workblk[x] = b
    for q = queue
        sendblk!(q, b)
    end
    b
end

event_genBlk! (generic function with 1 method)

In [7]:
function event_arrivalBlk!(q::Queue, i::Int, workblk::Dict{Minor,AbstractBlk})
    x = getreceiver(q)
    b = receiveblk!(q, i)
    prev = workblk[x]
    if getid(prev) < getid(b)
        workblk[x] = b
    end
    b
end

event_arrivalBlk! (generic function with 1 method)

In [8]:
function rates(u1, u2, queue, lambda, mu)
#    u1 = sort(collect(keys(lambda)), by=x->x.label)
    r1 = [lambda[x] for x = u1]
#    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    r2 = [mu[x] * getlength(queue[x]) for x = u2]
    return r1, r2
end

rates (generic function with 1 method)

In [9]:
a = Minor("A")
b = Minor("B")
c = Minor("C")

Minor("C")

In [10]:
lambda = Dict(
    a => 1/10,
    b => 1/10,
    c => 1/10,
)

mu = Dict(
    (a,b) => 1/0.1,
    (a,c) => 1/0.1,
    (b,a) => 1/0.1,
    (b,c) => 1/0.1,
    (c,a) => 1/0.1,
    (c,b) => 1/0.1,
)

N = 1

1

In [11]:
rng = MersenneTwister(1234)
Aworkblk = []
Ablocks = []
@time for k in 1:N
    blocks = Vector{Blk}()

    events = 0
    u1 = sort(collect(keys(lambda)), by=x->x.label)
    u2 = sort(collect(keys(mu)), by=x->(x[1].label, x[2].label))
    workblk = Dict{Minor,AbstractBlk}(a=>Genesis(), b=>Genesis(), c=>Genesis())
    qAtoB = Queue(a, b)
    qAtoC = Queue(a, c)
    qBtoA = Queue(b, a)
    qBtoC = Queue(b, c)
    qCtoA = Queue(c, a)
    qCtoB = Queue(c, b)
    queue = Dict(
        (a,b)=>qAtoB,
        (a,c)=>qAtoC,
        (b,a)=>qBtoA,
        (b,c)=>qBtoC,
        (c,a)=>qCtoA,
        (c,b)=>qCtoB,
    )
    input = Dict(
        a=>[qAtoB, qAtoC],
        b=>[qBtoA, qBtoC],
        c=>[qCtoA, qCtoB],
    )

    ctime = 0.0
    while ctime < 60*24*30*12*10 # 24 hours
        r1, r2 = rates(u1, u2, queue, lambda, mu)
        etime1 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r1]
        etime2 = [(r != 0) ? rand(rng, Exponential(1/r)) : Inf for r = r2]
        m1, i1 = findmin(etime1)
        m2, i2 = findmin(etime2)
        if m1 < m2
            ctime += m1
            event_genBlk!(u1[i1], workblk, input[u1[i1]], blocks)
        else
            ctime += m2
            v = rand(rng, DiscreteUniform(1,getlength(queue[u2[i2]])))
            event_arrivalBlk!(queue[u2[i2]], v, workblk)
        end
        events += 1
    end
    push!(Aworkblk,workblk)
    push!(Ablocks,blocks)
end

 49.944777 seconds (189.23 M allocations: 5.901 GiB, 9.25% gc time)


In [12]:
Aworkblk[1]

Dict{Minor,AbstractBlk} with 3 entries:
  Minor("A") => (1525757, "A")
  Minor("C") => (1525757, "A")
  Minor("B") => (1525758, "B")

In [13]:
Achains = []
for i in 1:N
    workblk = Aworkblk[i]
    blk = workblk[a]
    chains = []
    push!(chains, blk)
    while blk != Genesis()
        blk = blk.prev
        push!(chains, blk)
    end
    push!(Achains,chains)
end

In [14]:
Ainvchains = []
for i in 1:N
    chains = Achains[i]
    invchains = reverse(chains)
    push!(Ainvchains,invchains)
end

## 後続にブロックが付かない場合

In [78]:
#blocksにはあるがchainsにないものをカウントする
Ahaki = []
@time for h in 1:N
    haki = 0
    for i in 2:length(Ainvchains[h])
        if (Ainvchains[h][i].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("C"))
            for j in 1:length(Ablocks[h])
                if (Ablocks[h][j].minor == Minor("A")) && (Ablocks[h][j].index == Ainvchains[h][i].index)
                    haki += 1
                end
            end
        end
    end
   push!(Ahaki,haki)
    if h%20 == 0
        print(h,"ok")
    end
end

20ok40ok60ok80ok100ok5126.094939 seconds (96.93 G allocations: 1.729 TiB, 4.72% gc time)


## 後続にブロックが1つ以上付いた場合

In [ ]:
#blocksで連番かつchainsにない場合は二つ以上同時に破棄されていると考える
Ahaki = []
@time for h in 1:N
    haki = 0
    count = 0
    for i in 3:length(Ainvchains[h])-1
        if i%1000 == 0
            count += 1
            print(count,",")
        end
#        if Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i-1].minor == Minor("B")
#            for j in 1:length(Ablocks[h])
#                if (Ablocks[h][j].minor == Minor("A")) && (Ablocks[h][j].index == Ainvchains[h][i-1].index)
#                    for k in 1:length(Ablocks[h])
#                        if (Ablocks[h][k].minor == Minor("A")) && (Ablocks[h][k].index == Ainvchains[h][i].index)
#                            haki += 1
#                        end
#                    end
#                end
#            end
        if (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+1].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+1].minor == Minor("C")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+1].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+1].minor == Minor("C"))
            if (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+2].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("B") && Ainvchains[h][i+2].minor == Minor("C")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+2].minor == Minor("B")) || (Ainvchains[h][i].minor == Minor("C") && Ainvchains[h][i+2].minor == Minor("C"))
                for j in 1:length(Ablocks[h])
                    if (Ablocks[h][j].minor == Minor("A")) && (Ablocks[h][j].index == Ainvchains[h][i].index)
                        for k in 1:length(Ablocks[h])
                            if (Ablocks[h][k].minor == Minor("A")) && (Ablocks[h][k].index == Ainvchains[h][i+1].index)
                                if (Ablocks[h][k].minor == Minor("A")) && (Ablocks[h][k].index == Ainvchains[h][i+2].index)
                                    haki += 1
                                end
                            end
                        end
                    end
                end
            end
        end
    end
    push!(Ahaki,haki)
    if h%20 == 0
        print(h,"ok")
    end
end

1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,27

In [ ]:
AblkA = []
for i in 1:N
    blkA = 0
    for j in 1:length(Ablocks[i])
        if Ablocks[i][j].minor == Minor("A")
            blkA += 1
        end
    end
    push!(AblkA,blkA)
end

In [ ]:
Aresult = []
for i in 1:N
    result = Ahaki[i] / AblkA[i]
    push!(Aresult,result)
end

## 平均

In [ ]:
sum = 0
for i in 1:N
    sum += Aresult[i]
end
ave = sum / N

## 分散

In [ ]:
sq = 0
for i in 1:N
    sq += (Aresult[i]-ave)^2
end
var = sq / N

In [ ]:
sta = sqrt(var)

## 信頼区間(95%)

In [ ]:
sita = ave - 1.9842 * (1/10) * sta
ue = ave + 1.9842 * (1/10) * sta
print("[",sita,",",ue,"]")

In [ ]:
e = 0
for i in 1:N
    if Aresult[i] < sita && Aresult[i] > ue
        e += 1
    end
end
e

In [ ]:
Ahaki

In [18]:
open("chains.txt","w") do out #ファイルに書く
    print(out,chains)
end

In [19]:
open("blocks.txt","w") do out #ファイルに書く
    print(out,blocks)
end